> What I cannot create, I do not understand.  
> \- Richard Feynman

I've probably learned RSA at least five times and yet I'd struggle to
come up with many of the details on the spot. I could tell you that it
has to do with primes and the difficulty in factoring large numbers,
but that's about as far as I'd get.

In an effort to actually understand and appreciate the algorithm, I
wanted to implement it myself. The result is a comically primitive
implementation and a security person's nightmare. Still, it
was a fun exercise and it turned out to be vastly more interesting
and full of surprises than I'd expected.

In [ ]:
from random import randint

def random_n_bit_int(n: int) -> int:
    """Returns a random integer of n bits"""
    return randint(2**(n-1), (2**n)-1)

In [2]:
def is_prime(n: int, k: int = 20) -> bool:
    """Uses Fermat's little theorem"""
    test_vals = [randint(1, n-1) for _ in range(k)]
    return all(((a**(n-1) % n) == 1) for a in test_vals)

In [3]:
def random_n_bit_prime(n: int) -> int:
    """Generates a random prime number of length n bits"""
    rand_num = random_n_bit_int(n)
    while not is_prime(rand_num):
        rand_num = random_n_bit_int(n)

    return rand_num

In [4]:
import math

def is_coprime(a: int, b: int) -> bool:
    return math.gcd(a, b) == 1

In [5]:
import itertools

def find_coprime(n: int) -> int:
    """Returns first odd number coprime with n"""
    for odd_num in itertools.count(3, 2):
        if is_coprime(odd_num, n):
            return odd_num

In [6]:
from typing import Tuple

def extended_euclid(a: int, b: int) -> Tuple[int, int, int]:
    """Returns x, y, d such that a*x + b*y = d"""
    if b == 0:
        return (1, 0, a)

    a_over_b, a_mod_b = divmod(a, b)
    x, y, d = extended_euclid(b, a_mod_b)
    return (y, x - (a_over_b*y), d)

def compute_modular_inverse(a: int, p: int) -> int:
    """Computes a^1 mod p == 1 using extended euclid method"""
    x, y, _ = extended_euclid(a, p)
    return x % p

In [7]:
from collections import namedtuple

PublicKey = namedtuple("PublicKey", ["n", "e"])

In [8]:
class User:

    def __init__(self, name: str):
        self.name = name
        p, q = random_n_bit_prime(8), random_n_bit_prime(8)
        n = p*q
        phi_n = (p-1) * (q-1)
        e = find_coprime(phi_n)
        self._private_key = compute_modular_inverse(e, phi_n)
        self.public_key = PublicKey(n, e)

    def _find_secret_key(self, e, phi_n) -> int:
        for odd_num in itertools.count(3, 1):
            if (odd_num*e) % phi_n == 1:
                return odd_num

    def send_message(self, user, message: int) -> int:
        """Send a message to other user"""
        print(f"{self.name} sending secret message {message}")
        encrypted_msg = (message**user.public_key.e) % user.public_key.n
        print(f"{self.name} sending encrypted message: {encrypted_msg}")
        user.receive_message(encrypted_msg)

    def receive_message(self, encrypted_msg: int):
        """Receive encrypted message"""
        print(f"{self.name} received encrypted message: {encrypted_msg}")
        decrypted_msg = (encrypted_msg**self._private_key) % self.public_key.n
        print(f"{self.name} decrypted message: {decrypted_msg}")

In [9]:
Alice = User("Alice")
Bob = User("Bob")

In [10]:
Alice.send_message(Bob, 42)

Alice sending secret message 42
Alice sending encrypted message: 5656
Bob received encrypted message: 5656
Bob decrypted message: 42
